In [ ]:
'''
FermiPy Pipeline
==============================================
Created by Tai Withers, 08/2021
Modified by Tai Withers on 31/08/2021
==============================================

'''

In [1]:
# general
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, date, timedelta, time
from dateutil.relativedelta import relativedelta
import os

# email handling
import imaplib
import email
from email.header import decode_header
import webbrowser
import smtplib
from email import encoders
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
# https://www.thepythoncode.com/article/reading-emails-in-python
# https://www.thepythoncode.com/article/sending-emails-in-python-smtplib

# accessing gcn archives
from requests_html import HTMLSession
import requests

# using catalog
import xml.etree.ElementTree as ET

# making config files
import yaml

# accessing database
import mariadb
import sys
# https://mariadb.com/resources/blog/how-to-connect-python-programs-to-mariadb/

# fermi analysis
# from fermipy.gtanalysis import GTAnalysis
# some weird stuff with matplotlib versions
# roiplotter
# https://github.com/FermiSummerSchool/fermi-summer-school/tree/master/Likelihood_Advanced

# Send GCN Archives to Email

In [2]:
def get_archives(print_archives=False):
    '''
    Accesses the GCN Archives pages and returns a list of all of the links to individual notices
    '''
    session = HTMLSession()
    notices = ['https://gcn.gsfc.nasa.gov/notices_amon_icecube_cascade/135196_31863828.amon','https://gcn.gsfc.nasa.gov/notices_amon_icecube_cascade/135136_69399950.amon','https://gcn.gsfc.nasa.gov/notices_amon_icecube_cascade/134912_80902176.amon']
    # cascade contains test events so I added the real ones manually

    link_list= ['https://gcn.gsfc.nasa.gov/amon_hese_events.html',
                'https://gcn.gsfc.nasa.gov/amon_ehe_events.html',
                'https://gcn.gsfc.nasa.gov/amon_icecube_gold_bronze_events.html',
                'https://gcn.gsfc.nasa.gov/amon_nu_em_coinc_events.html']

    for link in link_list:
        link_session = session.get(link)
        for i in link_session.html.absolute_links:
            if '.amon' in i: notices.append(i)
    
    if print_archives: 
        for i in notices: print(i)
            
    return notices

def mail_gcn(notices, print_confirmation=False):
    '''
    Takes a list of links and emails the contents of each link to ******@gmail.com 
    '''
    
    email = "******@gmail.com"
    password = "******"
    subject = "GCN Archives"
    
    num_sent = 0

    def create_mail(FROM, TO, subject, body):
        msg = MIMEMultipart("alternative")# initialize the message 
        # set the sender, reciever, subject, body
        msg["From"] = FROM
        msg["To"] = TO
        msg["Subject"] = subject
        msg.attach(MIMEText(body,"plain"))
        return msg

    def send_mail(email, password, FROM, TO, msg):
        server = smtplib.SMTP("smtp.gmail.com", 587) # initialize the SMTP server
        server.starttls() # connect to the SMTP server as TLS mode (secure) and send EHLO        
        server.login(email, password)# login to the account using the credentials        
        server.sendmail(FROM, TO, msg.as_string())# send the email        
        server.quit()# terminate the SMTP session
    
    for i in notices:
        text = requests.get(i).text
        body = [] # list to handle single or updated notices
        
        if text.count('TITLE')>1: # deal with notice updates
            def find_nth(haystack, needle, n):
                start = haystack.find(needle)
                while start >= 0 and n > 1:
                    start = haystack.find(needle, start+len(needle))
                    n -= 1
                return start
            second_title_index = find_nth(text,'TITLE',2)
            body.append(text[:second_title_index]) # first notice
            body.append(text[second_title_index:]) # update
        else:
            body.append(text) # singular notice
        
        for f in body: # send the email
            msg = create_mail(email, email, subject, f)
            send_mail(email, password, email, email, msg)
        
        if print_confirmation: print(i,len(body), "Sent")
        num_sent+= len(body)
        
    if print_confirmation: print('Done', num_sent)

In [4]:
# notices = get_archives()
# notices_1 = notices[:50]
# notices_2 = notices[50:]

# if you don't split it up like this the gmail server gets mad and kicks you out
# total: 146 emails

In [5]:
# mail_gcn(notices_1,print_confirmation=True)

In [ ]:
# mail_gcn(notices_2,print_confirmation=True)

# Email Parsing

In [7]:
def get_gmail (username, password, num_messages=0, print_inbox=False, subject_filter="", sender_filter=""):   
    '''
    Takes email credentials and subject/sender filters
    Returns list of all emails in the inbox
    Each list element is formatted [subject, sender, message]
    '''
    
    imap = imaplib.IMAP4_SSL("imap.gmail.com") # create an IMAP4 class with SSL 
    imap.login(username, password) # authenticate
    
    status, messages = imap.select("INBOX")
    messages = int(messages[0]) # total number of emails
    
    if num_messages==0: num_messages=messages

    inbox = [] # [subject, from, message]

    for i in range(messages, messages-num_messages, -1):
        res, msg = imap.fetch(str(i), "(RFC822)") # fetch the email message by ID
        this_message = {}

        for response in msg:
            if isinstance(response, tuple):
                msg = email.message_from_bytes(response[1]) # parse a bytes email into a message object

                # decode the email subject    
                subject, encoding = decode_header(msg["Subject"])[0] 
                if isinstance(subject, bytes): subject = subject.decode(encoding)

                # decode email sender
                From, encoding = decode_header(msg.get("From"))[0]
                
                if isinstance(From, bytes): From = From.decode(encoding)
                if From=='******@gmail.com': # from has multiple formats, we want email only
                    sender=From
                else:
                    sender = From[From.index('<')+1:From.index('>')] # just email address
                
                    
                # EMAIL FILTERING 
                subject_match = subject_filter=="" or subject==subject_filter
                sender_match = sender_filter=="" or sender==sender_filter
                    
                if subject_match and sender_match:
                    if print_inbox: print("Subject:", subject, "\nFrom:", From)
                    this_message['Subject'] = subject
                    this_message['Sent From'] = sender

                    for part in msg.walk(): # iterate over email parts
                        # extract content type of email
                        content_type = part.get_content_type()
                        content_disposition = str(part.get("Content-Disposition"))
                        try: body = part.get_payload(decode=True).decode() # get the email body
                        except: pass

                        # print text/plain emails and skip attachments
                        if content_type == "text/plain" and "attachment" not in content_disposition:
                            if print_inbox: print(body)
                            this_message['Message'] = body

                    if print_inbox: print("="*100)
                        
        if this_message: inbox.append(this_message) # only append if not empty
    
    # close the connection and logout
    imap.close()
    imap.logout()
        
    return inbox

def numerical_month(str_mon):
    '''
    Turn a month abbreviation into a numerical index
    '''
    return ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'].index(str_mon) + 1

def general_scraper (msg, print_alert=False):
    '''
    Pull relevant alert information from a string and return it in a dictionary
    '''
    
    # some pieces rely on the list order of values
    notice_type = 'NOTICE_TYPE'
    ra, dec = 'SRC_RA', 'SRC_DEC'
    date, time = 'DISCOVERY_DATE', 'DISCOVERY_TIME',
    energy = 'ENERGY'
    
    alert = {}
    
    # get notice date and time
    dt_string = msg[msg.index('NOTICE_DATE')+18+4:msg.index('NOTICE_DATE')+18+22]
    day = int(dt_string[:2])
    month = int(numerical_month(dt_string[3:6]))
    year = int('20'+dt_string[7:9])
    hour = int(dt_string[10:12])
    minute = int(dt_string[13:15])
    second = int(dt_string[16:18])
    alert['Notice Date'] = datetime(year,month,day,hour=hour,minute=minute,second=second)
  
    # get event number and run number
    run_string = msg[msg.index("RUN_NUM")+18:]
    alert['Run'] = str(run_string[:run_string.index('\r')])
    event_string = msg[msg.index("EVENT_NUM")+18:]
    alert['Event'] = str(event_string[:event_string.index('\r')]) 
    
    # determine notice type
    n = msg.index(notice_type)
    alert['Type'] = msg[n+18 : n + msg[n:].index('\n')-1]
    
    # get RA
    ra_string = msg[msg.index(ra)+18:msg.index(dec)-2] # relies on dec
    J2000_splitter = ra_string.index("(J2000)")
    current_splitter = ra_string.index("(current)")
    s = ra_string[:J2000_splitter-1]
    alert['RA_J2000'] = float(s[:s.index('d')])
    s = ra_string[:current_splitter-1]
    alert['RA_current'] = float(ra_string[J2000_splitter+28:current_splitter-1][:s.index('d')])
    alert['RA_1950'] = float(ra_string[current_splitter+30:-7][:s.index('d')])
    
    # get DEC
    dec_string = msg[msg.index(dec)+18:msg.index('SRC_ERROR')-2] # relies on error
    J2000_splitter = dec_string.index("(J2000)")
    current_splitter = dec_string.index("(current)")
    s = dec_string[:J2000_splitter-1]
    alert['DEC_J2000'] = float(s[:s.index('d')])
    s = dec_string[:current_splitter-1]
    alert['DEC_current'] = float(dec_string[J2000_splitter+28:current_splitter-1][:s.index('d')])
    alert['DEC_1950'] = float(dec_string[current_splitter+30:-7][:s.index('d')])
    
    # get date
    alert['Event Date'] = msg[msg.index("DOY;")+7:msg.index("(yy/mm/dd)")-1]
    
    # get time
    alert['Event Time'] = msg[msg.index('SOD')+5:msg.index("} UT")]
    
    # error
    err_string = msg[msg.index('SRC_ERROR')+18:msg.index("50% containment")+16]
    n = err_string.index('[')
    if '90%' in err_string: # if lists both errors
        # 90%
        e90_string = err_string[:n] + err_string[n+1:err_string.index('radius')-1]
        if 'arcmin' in e90_string:
            alert['Error 90%'] = arcmin_to_deg(float(e90_string[:e90_string.index('arcmin')-1]))
        elif 'deg' in e90_string:
            alert['Error 90%'] = float(e90_string[:e90_string.index('deg')-1]) 
        # 50%
        err_string = msg[msg.index('SRC_ERROR50')+18:msg.index("50% containment")+16]
        n = err_string.index('[')
        e50_string = err_string[:n] + err_string[n+1:err_string.index('radius')-1]
        if 'arcmin' in e50_string:
            alert['Error 50%'] = arcmin_to_deg(float(e50_string[:e50_string.index('arcmin')-1]))
        elif 'deg' in e50_string:
            alert['Error 50%'] = float(e50_string[:e50_string.index('deg')-1])
    else: # only lists 50%
        if 'arcmin' in err_string:
            alert['Error 50%'] = arcmin_to_deg(float(err_string[:err_string.index('arcmin')-1]))
        elif 'deg' in err_string:
            alert['Error 50%'] = float(err_string[:err_string.index('deg')-1])
        alert['Error 90%'] = 0 # none would be more correct but this will allow database to be floats
        
      
    # Not every alert lists energy (HESE and NuEM consistently, encountered EHE w/o energy)
    try:
        energy_string = msg[msg.index(energy)+18:msg.index('eV]')+3]
        if 'MeV' in energy_string:
            alert['Energy'] = float(energy_string[:energy_string.index('MeV')-1])*10e6
        elif 'GeV' in energy_string:
            alert['Energy'] = float(energy_string[:energy_string.index('GeV')-1])*10e9
        elif 'TeV' in energy_string:
            alert['Energy'] = float(energy_string[:energy_string.index('TeV')-1])*10e12
    except ValueError:
        alert['Energy'] = 0 # none would be more correct but this will allow database to be floats
        
    # NuEM lists time
    if 'Neutrino-EM' in alert['Type']:
        alert['Time'] = float(msg[msg.index('DELTA_T')+18:msg.index('[sec]')])
    else: 
        alert['Time'] = 0 # none would be more correct but this will allow database to be floats
        
    # NuEM, Gold, and Bronze list false alarm rate
    if len(set(alert['Type']) & {'Neutrino-M', 'Gold', 'Bronze'}) > 0:
        alert['False Alarm Rate']: float(msg[msg.index('FAR')+18:msg.index('[yr^-1]')]) 
    else: alert['False Alarm Rate'] = 0 # none would be more correct but this will allow database to be floats
        
    # Gold and Bronze list signalness
    if 'Gold' in alert['Type'] or 'Bronze' in alert['Type']:
        alert['Signalness'] = float(msg[msg.index('SIGNALNESS')+18:msg.index('[dn]')])
    else: alert['Signalness'] = 0 # none would be more correct but this will allow database to be floats
    
    # printing
    if print_alert:
        for i in alert.keys():
            print(i+":\t"+alert[i])

    return alert

def get_catalog(xml):
    '''
    Takes an xml fermi catalog and returns a list of dictionaries
    Each source dictionary contains the name, RA/DEC, and flux
    '''
    sources = []
    
    root = ET.parse(xml).getroot() # access xml tree
    for child in root: # for element in catalog
        source = {}
        if child.attrib['type']=='PointSource': # point sources only
            source['Name'] = child.attrib['name'] 
            source['RA'] = float(child[1][0].attrib['value'])
            source['DEC'] = float(child[1][1].attrib['value'])
            source['Flux'] = float(child.attrib['Energy_Flux100'])
        if source: sources.append(source) # append only if not empty

    return sources

def find_nearby_sources(source_list, ra, dec, radius):
    '''
    Given a list of sources, and point in the sky, finds all sources within a given radius
    And returns them in  a list
    '''
    nearby_sources = []
    
    for source in source_list:
        distance_squared = (source['RA']-ra)**2 + (source['DEC']-dec)**2 
        if distance_squared <= radius**2: nearby_sources.append(source)# if w/i error radiues
            
    return nearby_sources

def arcmin_to_deg(arcmin):
    return arcmin/60

def scrape_inbox_msgs(inbox, print_alerts=False):
    '''
    Runs the scraper function on the results of the inbox function
    Returns a list of alert dictionaries
    '''
    alerts = []
    for mail in inbox:
        alert = general_scraper(mail['Message'], print_alert=print_alerts)
        alerts.append(alert)
    return alerts

def add_nearby_sources_to_alerts(alerts, source_list):
    '''
    Runs the find nearby sources function 
    '''
    for event in alerts:
        if 'Error 90%' in event.keys():
            event['Sources in 90% Region'] = find_nearby_sources(catalog,event['RA_J2000'],event['DEC_J2000'],event['Error 90%'])

        if 'Error 50%' in event.keys():
            event['Sources in 50% Region'] = find_nearby_sources(catalog,event['RA_J2000'],event['DEC_J2000'],event['Error 50%'])
            
    return alerts

def print_alerts(alerts):
    for event in alerts:
        for key in event.keys():
            print(key+':\t',event[key])
        print('='*100)

In [16]:
inbox = get_gmail('******@gmail.com', '******', subject_filter="GCN Archives")

In [17]:
catalog = get_catalog('gll_psc_v26.xml')
# could try using the make4FGL.py script and the fits file which may be more recent than the xml file
# https://fermi.gsfc.nasa.gov/ssc/data/access/lat/10yr_catalog/

In [18]:
alerts = scrape_inbox_msgs(inbox)

# Database

In [19]:
db_user = ******
db_pwd = ******
db_host = ******
db_port = ******
db_name = 'fermi'

In [20]:
def db_connect(username, password, host, port, database, print_confirmation=False):
    '''
    Connect to a database
    '''
    try:
        conn = mariadb.connect(user=username, password=password, host=host, port=port, database=database)
        if print_confirmation: print("Connected to",database)
    except mariadb.Error as e:
        print(f"Error conntecting to MariaDB Platform: {e}")
        sys.exit(1)

    cur = conn.cursor()
    
    return cur, conn

def clear_table(cursor,table):
    cursor.execute("DROP TABLE IF EXISTS "+table)
    
def create_alerts_table(cursor):
    string = '''CREATE TABLE IF NOT EXISTS alerts (
    db_ID INT PRIMARY KEY AUTO_INCREMENT,
    Notice_Date DATETIME,
    run INT, event INT,
    type VARCHAR(100),
    RA_J2000 FLOAT, RA_current FLOAT, RA_1950 FLOAT,
    DEC_J2000 FLOAT, DEC_current FLOAT, DEC_1950 FLOAT,
    Event_Date DATE, Event_Time TIME,
    Error_90 FLOAT, Error_50 FLOAT,
    Energy FLOAT, 
    Time FLOAT, False_Alarm_Rate FLOAT, Signalness FLOAT, Analysis INT DEFAULT 0)'''
    
    cursor.execute(string)
    
def describe_alerts_table(cursor):
    cur.execute("DESCRIBE alerts")

    print("Field \t\t Type \t\t Null \t Key \t Default \t Extra")
    for i in cur:
        print(i[0],' '*(15-len(i[0])),i[1],' '*(14-len(i[1])),i[2],'\t',i[3],'\t',i[4],'\t\t',i[5])
        
def add_alerts(cursor, alerts, print_confirmation=False):
    for alert in alerts:
        # check if duplicate
        run_event_filter = 'WHERE run='+str(alert['Run'])+' AND event='+str(alert['Event'] + ' AND type="'+alert['Type']+'"')
        matches = get_from_table(cursor,filter_string=run_event_filter) # list w/ 0 or 1 element(s)
        
        if len(matches)>0: # if duplicate, check if newer
            if matches[0]['Notice_Date']<alert['Notice Date']: # if newer, overwrite
                overwrite_string = '''UPDATE alerts 
                    SET Notice_Date='''+str(alert['Notice Date'])+'''
                        RA_J2000='''+str(alert['RA_J2000'])+'''
                        RA_current='''+str(alert['RA_current'])+'''
                        RA_1950='''+str(alert['RA_1950'])+'''
                        DEC_J2000='''+str(alert['DEC_J2000'])+'''
                        DEC_current='''+str(alert['DEC_current'])+'''
                        DEC_1950='''+str(alert['DEC_1950'])+'''
                        Event_Date='''+str(alert['Event Date'])+'''
                        Event_Time='''+str(alert['Event Time'])+'''
                        Error_90='''+str(alert['Error 90%'])+'''
                        Error_50='''+str(alert['Error 50%'])+'''
                        Energy='''+str(alert['Energy'])+'''
                        Time='''+str(alert['Time'])+'''
                        False_Alarm_Rate='''+str(alert['False Alarm Rate'])+'''
                        Signalness='''+str(alert['Signalness'])+'''
                    WHERE db_ID='''+str(matches[0]['db_ID'])
                try:
                    cursor.execute(overwrite_string)
                    if print_confirmation: print(matches[0]['db_ID'], "Overwritten successfully")
                except mariadb.Error as e:
                    print(f"Error: {e}")
         
        else: # not a duplicate, add a new entry
            alert_insert_values = tuple([str(alert[key]) for key in alert.keys()])
            insert_string = '''INSERT INTO alerts (Notice_Date,run,event,type,
                RA_J2000,RA_current,RA_1950,
                DEC_J2000,DEC_current,DEC_1950,
                Event_Date,Event_Time,
                Error_90,Error_50,
                Energy, Time, False_Alarm_Rate, Signalness) 
                VALUES '''+str(alert_insert_values)
            
            try: 
                cursor.execute(insert_string)
                if print_confirmation: print(alert_insert_values[1], alert_insert_values[2],"Written successfully")
            except mariadb.Error as e:
                print('For alert with run',alert_insert_values[1],'and event',alert_insert_values[2])
                print('For alert with run',alert_insert_values[1],'and event',alert_insert_values[2],f"Error: {e}")
                
def get_from_table(cursor,filter_string="",print_results=False):
    cursor.execute("SELECT * FROM alerts "+filter_string) # get database entries
    
    keys = ['db_ID','Notice_Date','run','event','type',
            'RA_J2000','RA_current','RA_1950','DEC_J2000','DEC_current','DEC_1950',
            'Event_Date','Event_Time','Error_90','Error_50','Energy', 'Time', 'False_Alarm_Rate', 'Signalness', 'Analysis']
    results = []
    
    for alert in cursor: # for every entry
        dict_alert = {keys[i]:alert[i] for i in range(len(keys))} # turn it into a dictionary
        results.append(dict_alert)
        if print_results:
            for key in keys:
                print(key,':\t',dict_alert[key])
            print('='*100,'\n')
    
    return results

def exit_db(connection):
    connection.commit()
    connection.close()

In [27]:
cur, con = db_connect(db_user,db_pwd,db_host,db_port,db_name)
# clear_table(cur,'alerts') # now we have duplicate checks don't need to delete table every time
# create_alerts_table(cur)
# describe_alerts_table(cur)
# add_alerts(cur,alerts)
all_alerts = get_from_table(cur,print_results=True)
exit_db(con)

db_ID :	 1
Notice_Date :	 2021-01-11 13:15:54
run :	 0
event :	 73310
type :	 AMON Neutrino-EM Coincidence 
RA_J2000 :	 162.34
RA_current :	 162.622
RA_1950 :	 161.669
DEC_J2000 :	 19.46
DEC_current :	 19.3484
DEC_1950 :	 19.7248
Event_Date :	 2021-01-11
Event_Time :	 13:06:41
Error_90 :	 0.669833
Error_50 :	 0.369833
Energy :	 0.0
Time :	 22742.2
False_Alarm_Rate :	 0.0
Signalness :	 0.0
Analysis :	 0

db_ID :	 2
Notice_Date :	 2021-05-15 00:43:32
run :	 0
event :	 85790
type :	 AMON Neutrino-EM Coincidence 
RA_J2000 :	 93.64
RA_current :	 93.9448
RA_1950 :	 92.9267
DEC_J2000 :	 14.66
DEC_current :	 14.6521
DEC_1950 :	 14.6759
Event_Date :	 2021-05-15
Event_Time :	 0:20:43
Error_90 :	 0.267667
Error_50 :	 0.149833
Energy :	 0.0
Time :	 22442.9
False_Alarm_Rate :	 0.0
Signalness :	 0.0
Analysis :	 0

db_ID :	 3
Notice_Date :	 2020-11-07 16:02:47
run :	 0
event :	 66291
type :	 AMON Neutrino-EM Coincidence 
RA_J2000 :	 140.2
RA_current :	 140.509
RA_1950 :	 139.456
DEC_J2000 :	 29.76
DE

RA_J2000 :	 242.58
RA_current :	 242.819
RA_1950 :	 241.99
DEC_J2000 :	 11.6099
DEC_current :	 11.5581
DEC_1950 :	 11.7394
Event_Date :	 2020-04-10
Event_Time :	 23:19:55
Error_90 :	 10.3698
Error_50 :	 6.50983
Energy :	 1100000000000000.0
Time :	 0.0
False_Alarm_Rate :	 0.0
Signalness :	 0.30535
Analysis :	 0

db_ID :	 22
Notice_Date :	 2019-07-12 03:11:47
run :	 132814
event :	 44222682
type :	 ICECUBE Astrotrack Bronze
RA_J2000 :	 76.4599
RA_current :	 76.7346
RA_1950 :	 75.7569
DEC_J2000 :	 13.06
DEC_current :	 13.0852
DEC_1950 :	 12.9932
Event_Date :	 2019-07-12
Event_Time :	 1:15:17
Error_90 :	 4.95983
Error_50 :	 3.04
Energy :	 1086700000000000.0
Time :	 0.0
False_Alarm_Rate :	 0.0
Signalness :	 0.30332
Analysis :	 0

db_ID :	 23
Notice_Date :	 2019-07-04 20:54:57
run :	 132792
event :	 60166398
type :	 ICECUBE Astrotrack Bronze
RA_J2000 :	 161.85
RA_current :	 162.117
RA_1950 :	 161.164
DEC_J2000 :	 27.1099
DEC_current :	 27.0066
DEC_1950 :	 27.3739
Event_Date :	 2019-07-04
Eve

Event_Time :	 15:05:31
Error_90 :	 1.08
Error_50 :	 0.569833
Energy :	 2142900000000000.0
Time :	 0.0
False_Alarm_Rate :	 0.0
Signalness :	 0.56208
Analysis :	 0

db_ID :	 42
Notice_Date :	 2020-01-17 14:25:18
run :	 133634
event :	 1410505
type :	 ICECUBE Astrotrack Bronze
RA_J2000 :	 116.24
RA_current :	 116.552
RA_1950 :	 115.459
DEC_J2000 :	 29.1299
DEC_current :	 29.0803
DEC_1950 :	 29.2513
Event_Date :	 2020-01-17
Event_Time :	 11:08:29
Error_90 :	 0.91
Error_50 :	 0.51
Energy :	 1083900000000000.0
Time :	 0.0
False_Alarm_Rate :	 0.0
Signalness :	 0.3754
Analysis :	 0

db_ID :	 43
Notice_Date :	 2020-11-15 14:14:08
run :	 134699
event :	 70289682
type :	 ICECUBE Astrotrack Gold
RA_J2000 :	 195.12
RA_current :	 195.387
RA_1950 :	 194.481
DEC_J2000 :	 1.3799
DEC_current :	 1.2678
DEC_1950 :	 1.6491
Event_Date :	 2020-11-15
Event_Time :	 2:07:26
Error_90 :	 1.29
Error_50 :	 0.709833
Energy :	 1773800000000000.0
Time :	 0.0
False_Alarm_Rate :	 0.0
Signalness :	 0.45973
Analysis :	 0


Analysis :	 0

db_ID :	 64
Notice_Date :	 2017-03-21 07:32:58
run :	 129307
event :	 80305071
type :	 AMON ICECUBE EHE 
RA_J2000 :	 98.3268
RA_current :	 98.5229
RA_1950 :	 97.7574
DEC_J2000 :	 -14.4861
DEC_current :	 -14.5001
DEC_1950 :	 -14.4472
Event_Date :	 2017-03-21
Event_Time :	 7:32:20
Error_90 :	 0.324667
Error_50 :	 0.0
Energy :	 1199800000000000.0
Time :	 0.0
False_Alarm_Rate :	 0.0
Signalness :	 0.0
Analysis :	 0

db_ID :	 65
Notice_Date :	 2016-08-06 12:22:10
run :	 128311
event :	 26552458
type :	 AMON ICECUBE EHE 
RA_J2000 :	 122.798
RA_current :	 123.01
RA_1950 :	 122.16
DEC_J2000 :	 -0.7331
DEC_current :	 -0.7833
DEC_1950 :	 -0.5836
Event_Date :	 2016-08-06
Event_Time :	 12:21:33
Error_90 :	 0.111167
Error_50 :	 0.0
Energy :	 0.0
Time :	 0.0
False_Alarm_Rate :	 0.0
Signalness :	 0.0
Analysis :	 0

db_ID :	 66
Notice_Date :	 2018-09-08 20:00:16
run :	 131475
event :	 34507973
type :	 AMON ICECUBE EHE 
RA_J2000 :	 145.773
RA_current :	 146.01
RA_1950 :	 145.139
DEC_J2000

DEC_J2000 :	 -32.0165
DEC_current :	 -32.1038
DEC_1950 :	 -31.7532
Event_Date :	 2016-08-14
Event_Time :	 21:45:54
Error_90 :	 1.48983
Error_50 :	 0.479833
Energy :	 0.0
Time :	 0.0
False_Alarm_Rate :	 0.0
Signalness :	 0.0
Analysis :	 0

db_ID :	 83
Notice_Date :	 2019-03-31 06:56:27
run :	 132379
event :	 15947448
type :	 AMON ICECUBE HESE 
RA_J2000 :	 355.635
RA_current :	 355.858
RA_1950 :	 355.06
DEC_J2000 :	 71.117
DEC_current :	 71.2238
DEC_1950 :	 70.8395
Event_Date :	 2019-03-31
Event_Time :	 6:55:43
Error_90 :	 8.9
Error_50 :	 1.6
Energy :	 0.0
Time :	 0.0
False_Alarm_Rate :	 0.0
Signalness :	 0.0
Analysis :	 0

db_ID :	 84
Notice_Date :	 2016-04-27 14:39:05
run :	 127853
event :	 67093193
type :	 AMON ICECUBE HESE 
RA_J2000 :	 239.664
RA_current :	 239.864
RA_1950 :	 239.053
DEC_J2000 :	 6.8528
DEC_current :	 6.807
DEC_1950 :	 6.9947
Event_Date :	 2016-04-27
Event_Time :	 5:52:32
Error_90 :	 8.9
Error_50 :	 1.6
Energy :	 0.0
Time :	 0.0
False_Alarm_Rate :	 0.0
Signalness :	 

In [28]:
alerts_with_nearby = add_nearby_sources_to_alerts(alerts,catalog)

# Config
for every alert which needs analysis
run analysis on every source in the 90% region (except cascade: 50%)

In [29]:
def needs_analysis(alerts):
    '''
    Figure out which alerts need to analysed
    '''
    to_be_analysed = []
    for i in alerts:
        if i['Analyis']==0:
            to_be_analysed.append(i)
    return to_be_analysed

def clean_text(text):
    # clean text for creating a folder
    string = ""
    for c in text:
        if c.isalnum() or c=='-' or c=='+': string+=c
        else: string+='_'
    return string

def make_config(alert,source):
    # should add a way to update tmax
    config = yaml.safe_load(open('defaultconfig.yaml'))
    
    # setup up outdir: type/alert/source
    type_folder = clean_text(alert['Type'].strip())
    alert_folder = clean_text(str(alert['Run'])+'_'+str(alert['Event']))
    source_folder = clean_text(source['Name'])
    # ensure directories exist
    if not os.path.isdir(type_folder): os.mkdir(type_folder)
    if not os.path.isdir(type_folder+'/'+alert_folder): os.mkdir(type_folder+'/'+alert_folder)
    if not os.path.isdir(type_folder+'/'+alert_folder+'/'+source_folder): os.mkdir(type_folder+'/'+alert_folder+'/'+source_folder)
    # set config outdir
    config['fileio'] = {'outdir' : type_folder+'/'+alert_folder+'/'+source_folder}
    
    # set fermipy target
    config['selection']['target'] = source['Name']
    
    # set fermipy start time
    fermi_start = datetime(year=2008, month=8, day=4, hour=15, minute=43, second=36)
    fermi_start_sec = 239557417
    t = time.fromisoformat(alert['Event Time']) # get event time
    event_dt = datetime.combine(alert['Event Date'],time=t) # combine event date/time
#     t = (alert['Event Time'].seconds) # get event time
#     event_dt = datetime.combine(alert['Event Date'],time=time(t//3600,(t%3600)//60,(t%3600)%60)) # combine event date/time
    pre_event_dt = event_dt - relativedelta(months=-6) # get 6mo before event
    dif = pre_event_dt - fermi_start # how long after fermi started
    config['selection']['tmin'] = fermi_start_sec + dif.seconds # convert to fermi seconds
    
    yaml.dump(config,open(type_folder+'/'+alert_folder+'/'+source_folder+'/config.yaml','w'))

In [32]:
print(all_alerts[18])
print('\n\n')
print(alerts_with_nearby[28])

{'db_ID': 19, 'Notice_Date': datetime.datetime(2019, 9, 23, 1, 56, 41), 'run': 133091, 'event': 81419, 'type': 'ICECUBE Astrotrack Gold', 'RA_J2000': 167.43, 'RA_current': 167.673, 'RA_1950': 166.815, 'DEC_J2000': -22.39, 'DEC_current': -22.4972, 'DEC_1950': -22.1186, 'Event_Date': datetime.date(2019, 9, 22), 'Event_Time': datetime.timedelta(seconds=34965), 'Error_90': 2.95, 'Error_50': 1.68983, 'Energy': 3.1139e+16, 'Time': 0.0, 'False_Alarm_Rate': 0.0, 'Signalness': 0.20165, 'Analysis': 0}



{'Notice Date': datetime.datetime(2019, 9, 23, 1, 56, 41), 'Run': '133091', 'Event': '81419', 'Type': 'ICECUBE Astrotrack Gold', 'RA_J2000': 167.43, 'RA_current': 167.6729, 'RA_1950': 166.8149, 'DEC_J2000': -22.39, 'DEC_current': -22.4972, 'DEC_1950': -22.1186, 'Event Date': '19/09/22', 'Event Time': '09:42:45.62', 'Error 90%': 2.95, 'Error 50%': 1.6898333333333333, 'Energy': 3.1139e+16, 'Time': 0, 'False Alarm Rate': 0, 'Signalness': 0.20165, 'Sources in 90% Region': [{'Name': '4FGL J1100.0-204

In [45]:
for py_alert in alerts_with_nearby:
    if 'Cascade' not in py_alert['Type']:
        for i in py_alert['Sources in 90% Region']:
            make_config(py_alert,i)
    else:
        for i in py_alert['Sources in 50% Region']:
            make_config(py_alert,i)

ValueError: Invalid isoformat string: '12:41:21.41'